In [1]:
import torch
print(torch.__version__)

1.2.0


In [2]:
from __future__ import print_function

import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision import models
import numpy
import dataset
import random
from utils import *
from VGG_gru import FERANet
# from tensorboardX import SummaryWriter
import argparse
import subprocess
import math

In [3]:
# os.environ['CUDA_VISIBLE_DEVICES'] = "1"

description='PyTorch Facial Expression'
batch_size = 1
epochs = 20 #120
length = 32
preInitial = True
# dataset = 'Emotiw'

In [4]:
trn= "BDItrain1.txt"
vl= "BDIval1.txt"
tst= "BDItest1.txt"

In [5]:
trainlist = "/home/dua/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+trn
vallist =  "/home/dua/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+vl
testlist =  "/home/dua/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+tst

In [6]:
print(trainlist)

/home/dua/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/BDItrain1.txt


In [7]:
path_d= "/home/dua/Desktop/GRU-test/emotion_classification-master/VGG_GRU/dataset.py"

In [8]:
if os.path.isfile != True:
	subprocess.call(["python", path_d])

In [9]:
# if os.path.isfile != True:
# 	subprocess.call(["python", "./TrainTestlist/"+dataset+"/getTraintest_"+dataset+".py"])

In [10]:
backupdir     = "/tmp/dua/GRU"
batch_size    = 1
# learning_rate = 0.00001
learning_rate = 0.0001

# accuracy = 0.
# best_accuracy = 0.
metric = AccumulatedAccuracyMetric()

####here for same result#####
num_workers   = 0
# torch.backends.cudnn.enabled = False
torch.manual_seed(1)
torch.cuda.manual_seed(1)
random.seed(1)
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.deterministic = True

In [11]:
if preInitial == True:
	model = FERANet()
	model = Initial(model)
else:
	model = FERANet()

start loading vgg16 pre-fera2013 model...
finish loading vgg16 pre-fera2013 model!


In [12]:
processed_batches = 0
kwargs = {'num_workers': num_workers, 'pin_memory': True}
model = model.cuda()


optimizer = optim.SGD(model.parameters(), lr=learning_rate , momentum=0.9, weight_decay= 0.00005)

loss_function = nn.CrossEntropyLoss()

use_Tensorboard = False

In [13]:
def train(epoch,optimizer):


	train_loader = torch.utils.data.DataLoader(
		dataset.listDataset(trainlist,length = length,
					   shuffle=True,
					   train=True,
					   dataset = dataset),
		batch_size=batch_size, shuffle=False, **kwargs)


	for param_group in optimizer.param_groups:
		train_learning_rate = float(param_group['lr'])

	logging('epoch %d, processed %d samples, lr %f' % (epoch, epoch * len(train_loader.dataset), train_learning_rate))

	running_loss = 0.0

	model.train()

	for batch_idx, (data, label) in enumerate(train_loader):


		data = data.squeeze(0)
		data = Variable(data).cuda()

		label = Variable(label.long()).cuda()
		label = label.squeeze(1)

		optimizer.zero_grad()

		output = model(data)

		loss = loss_function(output, label)

		running_loss += loss.data

		loss.backward()

		optimizer.step()

	if epoch %1 == 0:
		logging('Loss:{:.6f}'.format(running_loss))

In [14]:
def eval(epoch,metric):

	model.eval()

# 	global best_accuracy
# 	global accuracy

	metric.reset()
    
	accuracy = 0.
	best_accuracy = 0.
    
	test_loader = torch.utils.data.DataLoader(
# 		dataset.listDataset(vallist,length = length,
		dataset.listDataset(vallist,length = length,        
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)


	for batch_idx, (data, label) in enumerate(test_loader):

		data = data.squeeze(0)

		Batch,T,C,H,W = data.size()

		data = Variable(data,volatile=True).cuda()

		label = Variable(label.long(),volatile=True).cuda()
		label = label.squeeze(1)
# 		print('label')
		output = []
		for batch_index in range(Batch):
# 			print(batch_index)            
			output_feature = model(data[batch_index])

			output.append(output_feature)

		output = torch.mean(torch.cat(output), 0, keepdim=True)
        

		metric(output, label) 
# 		print(metric.value())

		accuracy, eval_loss = metric.value()
# 		print(accuracy)

        
	if accuracy >= best_accuracy:
		best_accuracy = accuracy
		print("saving accuracy is: ",accuracy)
		torch.save(model.state_dict(),'%s/model_%d.pkl' % (backupdir,epoch))

	logging("test accuracy: %f" % (accuracy))
	logging("best accuracy: %f" % (best_accuracy))

	return accuracy

In [15]:
def test(epoch,metric):

	model.eval()

# 	global best_accuracy
# 	global accuracy

	metric.reset()
    
	accuracy = 0.
	best_accuracy = 0.
    
	test_loader = torch.utils.data.DataLoader(
# 		dataset.listDataset(vallist,length = length,
		dataset.listDataset(testlist,length = length,        
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)


	for batch_idx, (data, label) in enumerate(test_loader):

		data = data.squeeze(0)

		Batch,T,C,H,W = data.size()

		data = Variable(data,volatile=True).cuda()

		label = Variable(label.long(),volatile=True).cuda()
		label = label.squeeze(1)
# 		print('label')
		output = []
		for batch_index in range(Batch):
# 			print(batch_index)            
			output_feature = model(data[batch_index])

			output.append(output_feature)

		output = torch.mean(torch.cat(output), 0, keepdim=True)
        

		metric(output, label) 
# 		print(metric.value())

		accuracy, eval_loss = metric.value()
# 		print(accuracy)

        
	if accuracy >= best_accuracy:
		best_accuracy = accuracy
# 		print("saving accuracy is: ",accuracy)
# 		torch.save(model.state_dict(),'%s/model_%d.pkl' % (backupdir,epoch))

	logging("test accuracy: %f" % (accuracy))
	logging("best accuracy: %f" % (best_accuracy))

	return accuracy

In [16]:
for epoch in range(1, epochs+1): 

	train(epoch,optimizer)
	with torch.no_grad():
		eval_accuary = eval(epoch,metric)

2020-09-22 19:34:44 epoch 1, processed 36 samples, lr 0.000100
2020-09-22 19:34:57 Loss:25.269423


/home/dua/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/dua/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


saving accuracy is:  0.8333333333333334
2020-09-22 19:39:16 test accuracy: 0.833333
2020-09-22 19:39:16 best accuracy: 0.833333
2020-09-22 19:39:16 epoch 2, processed 72 samples, lr 0.000100
2020-09-22 19:39:26 Loss:20.927383
saving accuracy is:  0.8333333333333334
2020-09-22 19:43:33 test accuracy: 0.833333
2020-09-22 19:43:33 best accuracy: 0.833333
2020-09-22 19:43:33 epoch 3, processed 108 samples, lr 0.000100
2020-09-22 19:43:44 Loss:12.868725
saving accuracy is:  0.8333333333333334
2020-09-22 19:47:47 test accuracy: 0.833333
2020-09-22 19:47:47 best accuracy: 0.833333
2020-09-22 19:47:47 epoch 4, processed 144 samples, lr 0.000100
2020-09-22 19:47:58 Loss:12.175969
saving accuracy is:  0.8333333333333334
2020-09-22 19:52:05 test accuracy: 0.833333
2020-09-22 19:52:05 best accuracy: 0.833333
2020-09-22 19:52:05 epoch 5, processed 180 samples, lr 0.000100
2020-09-22 19:52:16 Loss:9.763355
saving accuracy is:  0.8333333333333334
2020-09-22 19:56:22 test accuracy: 0.833333
2020-09-22

In [17]:
for epoch in range(1, epochs+1): 
	with torch.no_grad():
		eval_accuary = test(epoch,metric)

/home/dua/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/dua/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


2020-09-22 21:04:00 test accuracy: 0.750000
2020-09-22 21:04:00 best accuracy: 0.750000
2020-09-22 21:08:09 test accuracy: 0.750000
2020-09-22 21:08:09 best accuracy: 0.750000
2020-09-22 21:12:24 test accuracy: 0.750000
2020-09-22 21:12:24 best accuracy: 0.750000
2020-09-22 21:16:34 test accuracy: 0.750000
2020-09-22 21:16:34 best accuracy: 0.750000
2020-09-22 21:20:48 test accuracy: 0.750000
2020-09-22 21:20:48 best accuracy: 0.750000
2020-09-22 21:24:58 test accuracy: 0.750000
2020-09-22 21:24:58 best accuracy: 0.750000
2020-09-22 21:29:10 test accuracy: 0.750000
2020-09-22 21:29:10 best accuracy: 0.750000
2020-09-22 21:33:23 test accuracy: 0.750000
2020-09-22 21:33:23 best accuracy: 0.750000
2020-09-22 21:37:31 test accuracy: 0.750000
2020-09-22 21:37:31 best accuracy: 0.750000
2020-09-22 21:41:36 test accuracy: 0.750000
2020-09-22 21:41:36 best accuracy: 0.750000
2020-09-22 21:45:41 test accuracy: 0.750000
2020-09-22 21:45:41 best accuracy: 0.750000
2020-09-22 21:49:56 test accurac